In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import pandas_profiling as pp
import matplotlib.pyplot as plt
%matplotlib inline

import Conversion

import warnings
warnings.filterwarnings('ignore')

wd = '/Users/ewenwang/Documents/practice_data/conversion_rate/'

Data = Conversion.Data(wd, data_list=Conversion.new_data, is_all=False)
dtrain, dvalidation = Data.ValidationFE(validation_day=[24])

In [2]:
Algorithm = Conversion.Algorithm(dtrain=dtrain, dvalidation=dvalidation, target=Data.target, features=Data.predictors)
clf_bsline = Algorithm.Baseline()

Start training...
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[349]	valid_0's binary_logloss: 0.0851535
log loss:  0.08515348878413638


In [3]:
clf_sim_1 = Algorithm.simAnneal(alpha=0.75, n_trans=5, scoring='neg_log_loss')

simulating...

INFO: Number of possible iterations given cooling schedule: 160



[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.7min finished


2 T: 10.0, score: -0.0003, std: 0.0000, params: {'max_depth': '3.00', 'subsample': '0.67', 'colsample_bytree': '0.27'}


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  4.2min finished


3 T: 10.0, score: -0.0003, std: 0.0000, params: {'max_depth': '26.00', 'subsample': '0.67', 'colsample_bytree': '0.27'}


...

4 T: 10.0, score: -0.0003, std: 0.0000, params: {'max_depth': '29.00', 'subsample': '0.67', 'colsample_bytree': '0.27'}


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  4.2min finished


5 T: 10.0, score: -0.0003, std: 0.0000, params: {'max_depth': '24.00', 'subsample': '0.67', 'colsample_bytree': '0.27'}


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.7min finished


6 T: 10.0, score: -0.0003, std: 0.0000, params: {'max_depth': '24.00', 'subsample': '0.04', 'colsample_bytree': '0.27'}

best score: -0.000276 
log loss: 0.157058 
best parameters: {'max_depth': '26.00', 'subsample': '0.67', 'colsample_bytree': '0.27'}


In [34]:
clf_sim_2 = Algorithm.simAnneal(alpha=0.75, n_trans=5, scoring='roc_auc')

simulating...

INFO: Number of possible iterations given cooling schedule: 160



[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.9min finished


2 T: 10.0, score: 1.0000, std: 0.0000, params: {'max_depth': '16.00', 'subsample': '0.11', 'colsample_bytree': '0.23'}


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  7.0min finished


3 T: 10.0, score: 1.0000, std: 0.0000, params: {'max_depth': '16.00', 'subsample': '0.11', 'colsample_bytree': '0.79'}


...

4 T: 10.0, score: 1.0000, std: 0.0000, params: {'max_depth': '29.00', 'subsample': '0.11', 'colsample_bytree': '0.79'}


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 11.2min finished


5 T: 10.0, score: 1.0000, std: 0.0000, params: {'max_depth': '29.00', 'subsample': '0.11', 'colsample_bytree': '0.03'}


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.3min finished


6 T: 10.0, score: 1.0000, std: 0.0000, params: {'max_depth': '20.00', 'subsample': '0.11', 'colsample_bytree': '0.03'}

best score: 1.000000 
log loss: 0.136173 
best parameters: {'max_depth': '16.00', 'subsample': '0.11', 'colsample_bytree': '0.18'}


In [4]:
Data_25 = Conversion.Data(wd, Conversion.new_data, is_all=False)
dtrain, dvalidation = Data.ValidationFE(validation_day=[25])

In [28]:
prediction = pd.DataFrame(clf_sim_1.best_estimator_.predict_proba(dvalidation[Data.predictors])[:,1], columns=['predicted_score'])
prediction.head()

,predicted_score
0,0.000031
1,0.000099
2,0.000098
3,0.000100
4,0.000095


In [30]:
print(prediction.shape, dvalidation.shape)

(18371, 1) (18371, 88)


In [31]:
dvalidation = dvalidation.reset_index()

In [32]:
submission = pd.DataFrame(columns=['instance_id', 'predicted_score'])
submission['instance_id'] = dvalidation['instance_id']
submission['predicted_score'] = prediction['predicted_score']
submission.head()

,instance_id,predicted_score
0,2475218615076601065,0.000031
1,398316874173557226,0.000099
2,6586402638209028583,0.000098
3,1040996105851528465,0.000100
4,6316278569655873454,0.000095


In [33]:
submission.to_csv('result_3.txt', index=False, sep=' ')